In [36]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import os

from helper_functions import elementlines, nearestlines, theoretical_energy

%matplotlib qt

In [37]:
##### SEM Apreo March 2023 #####
path = '../../Masteroppgave/2023-03-08_EDS-Apreo/exports/'
aztec_file = '../../Masteroppgave/2023-03-08_EDS-Apreo/AZtec k-factors and quantification.xlsx'
df = pd.read_excel(aztec_file, sheet_name=4)

# drop the last 4 rows, which is Map1 and Map2
df = df.drop(df.index[-4:])

df['%at'] = df['%at'] /100

In [38]:
hs.eds.electron_range('Cu', 10, density='auto', tilt=0)

0.4593018852012333

In [39]:
hs.eds.electron_range('Cu', 20, density='auto', tilt=0)

1.4615656269399775

In [40]:
hs.eds.electron_range('Cu', 30, density='auto', tilt=0)

2.8766744984001607

In [41]:
hs.eds.electron_range?

Signature: hs.eds.electron_range(element, beam_energy, density='auto', tilt=0)
Docstring:
Returns the maximum electron range for a pure bulk material according to
the Kanaya-Okayama parameterziation.

Parameters
----------
element: str
    The element symbol, e.g. 'Al'.
beam_energy: float
    The energy of the beam in keV.
density: {float, 'auto'}
    The density of the material in g/cm3. If 'auto', the density of
    the pure element is used.
tilt: float.
    The tilt of the sample in degrees.

Returns
-------
Electron range in micrometers.

Examples
--------
>>> # Electron range in pure Copper at 30 kV in micron
>>> hs.eds.electron_range('Cu', 30.)
2.8766744984001607

Notes
-----
From Kanaya, K. and S. Okayama (1972). J. Phys. D. Appl. Phys. 5, p43

See also the textbook of Goldstein et al., Plenum publisher,
third edition p 72.
File:      c:\users\brynjar\miniconda3\envs\hspy\lib\site-packages\hyperspy\misc\eds\utils.py
Type:      function

In [43]:
hs.material.mass_absorption_mixture(elements=['Ga','Sb'], weight_percent=[50,50], energies='Ga_Ka')

106.44460851482819

In [44]:
hs.material.mass_absorption_mixture(elements=['Ga','Sb'], weight_percent=[50,50], energies=['Ga_La','Sb_La'])

array([3981.17897293,  449.61890897])

In [45]:
hs.material.mass_absorption_mixture(elements=['Ga','Sb'], weight_percent=[100,0], energies=1.3)

7073.223074770055

In [46]:
hs.material.mass_absorption_coefficient(element='Ga', energies=['Ga_La','Sb_La'])

array([1245.06325119,  520.18218321])

In [47]:
hs.material.mass_absorption_coefficient(element='Sb', energies=['Ga_La','Ga_Ka'])

array([6717.29469467,  173.73187746])

In [48]:
def add_name_x_kV(df):
    # add a name column to the dataframe
    df['Name'] = df['File']
    df['Name'] = df['Name'].str.replace('_', ' ')
    # df['Name'] = df['Name'].str.replace('GaAs', '(a)')
    # df['Name'] = df['Name'].str.replace('GaSb', '(b)')
    df['Name'] = df['Name'].str.replace('processTime', 'PT')
    df['Name'] = df['Name'].str.replace('scratched', 'scr')
    df['i'] = 0.0
    df['i/i_sum'] = 0.0

    # add kV to the dataframe
    kv_list = df['Name'].str.split(' ')
    for i in range(len(kv_list)):
        # find the index which contains 'kV'
        for j in range(len(kv_list[i])):
            if 'kV' in kv_list[i][j]:
                kv_index = j
        kv_list[i] = float(kv_list[i][kv_index][:2])
    df['kV'] = kv_list

    # add a x-axis for plotting
    df['x'] = 0
    x = 0
    for f in df['File'].unique():
        df.loc[df['File'] == f, 'x'] = x
        x += 1

    return df

df = add_name_x_kV(df)

In [49]:
# while working now, I do not need "i corr" "k-ratio"
df = df.drop(columns=['i corr', 'k-ratio'])

In [50]:
df.head(3)

,File,Element,%at,Line,Name,i,i/i_sum,kV,x
0,GaAs_05kV_25pA,As,0.5342,As_La,GaAs 05kV 25pA,0.0,0.0,5.0,0
1,GaAs_05kV_25pA,Ga,0.4658,Ga_La,GaAs 05kV 25pA,0.0,0.0,5.0,0
2,GaAs_15kV_25pA,As,0.5062,As_La,GaAs 15kV 25pA,0.0,0.0,15.0,1


In [51]:
def calculate_intensities(filename):

        # check if filename exists in path
    if filename + '.emsa' not in os.listdir(path):
        print(filename + '.emsa', 'not found in path')
        return
    
    df_s = df[df['File'] == filename]
    df_s = df_s.sort_values(by=['Element']) # alphabetically, as HS

    noise_slice = 0.65 # keV

    # make the spectrum and the model
    s = hs.load(path + filename + '.emsa', signal_type='EDS_TEM')
    elements = df_s['Element'].values
    s = s.isig[noise_slice:] # noise peak slice
    if s.metadata.Acquisition_instrument.TEM.beam_energy < 20:
        s = s.isig[:s.metadata.Acquisition_instrument.TEM.beam_energy]
    s.add_elements(elements)
    s.add_lines()
    m = s.create_model()
    m.fit()
    m.fit_background()

    lines = df_s['Line'].values

    line_intensities = []
    for l in lines:
        line_intensities.append(round(m[l].A.value))

    relative_intensities = []  # this is wt%, I think (according to Castaing's paper)
    for line_intens in line_intensities:
        relative_intensities.append(round(line_intens/sum(line_intensities), 3))
    
    df_s['i'] = line_intensities
    df_s['i/i_sum'] = relative_intensities

    df[df['File'] == filename] = df_s

# calculate_intensities('GaAs_30kV_50pA')

In [52]:
for filename in df['File'].unique():
    calculate_intensities(filename)

In [53]:
at_wt_list = []
# as the iteration is over all the rows, some operations are repeated, but it is not a big deal.
for i in range(len(df)):
    at_wt = hs.material.elements[df['Element'][i]].General_properties.atomic_weight
    at_wt_list.append(at_wt)
df['at_wt'] = at_wt_list
print('i/i_sum is wt%')

i/i_sum is wt%


# i/i_sum is wt%

### at%_1 = (wt%_1 / at_wt_1) / (wt%_1 / at_wt_1 + wt%_2 / at_wt_2)

https://www.plasmaterials.com/converting-atomic-percent-to-weight-percent-and-vice-versa/

In [54]:
elements_list = []
at_perc_list =[]
for xi in df['x'].unique():
    elements_list.append(df[df['x']==xi]['Element'].unique().tolist())
    elements_list.append(df[df['x']==xi]['Element'].unique().tolist()) # twice as there are two elements per sample

    wt = df[df['x']==xi]['i/i_sum'].tolist()
    at_wt = df[df['x']==xi]['at_wt'].tolist()

    # at%_1 = (wt%_1 / at_wt_1) / (wt%_1 / at_wt_1 + wt%_2 / at_wt_2)
    # https://www.plasmaterials.com/converting-atomic-percent-to-weight-percent-and-vice-versa/
    at_perc_list.append((wt[0]/at_wt[0]) / (wt[0]/at_wt[0] + wt[1]/at_wt[1]))
    at_perc_list.append((wt[1]/at_wt[1]) / (wt[0]/at_wt[0] + wt[1]/at_wt[1]))

df['Elements'] = elements_list
df['i: at%'] = at_perc_list
df.head(1)

,File,Element,%at,Line,Name,i,i/i_sum,kV,x,at_wt,Elements,i: at%
0,GaAs_05kV_25pA,As,0.5342,As_La,GaAs 05kV 25pA,13128.0,0.445,5.0,0,74.9216,"[As, Ga]",0.427317


In [71]:
df[['File', 'Element', 'Elements', 'Line', 'i', 'i/i_sum', '%at','i: at%']]

,File,Element,Elements,Line,i,i/i_sum,%at,i: at%
0,GaAs_05kV_25pA,As,"[As, Ga]",As_La,13128.0,0.445,0.5342,0.427317
1,GaAs_05kV_25pA,Ga,"[As, Ga]",Ga_La,16365.0,0.555,0.4658,0.572683
2,GaAs_15kV_25pA,As,"[As, Ga]",As_La,61699.0,0.374,0.5062,0.357322
3,GaAs_15kV_25pA,Ga,"[As, Ga]",Ga_La,103207.0,0.626,0.4938,0.642678
4,GaAs_30kV_25pA,As,"[As, Ga]",As_Ka,83787.0,0.373,0.4841,0.356341
5,GaAs_30kV_25pA,Ga,"[As, Ga]",Ga_Ka,140854.0,0.627,0.5159,0.643659
6,GaAs_30kV_50pA,As,"[As, Ga]",As_Ka,179725.0,0.372,0.4831,0.355361
7,GaAs_30kV_50pA,Ga,"[As, Ga]",Ga_Ka,303060.0,0.628,0.5169,0.644639
8,GaSb_05kV_50pA,Ga,"[Ga, Sb]",Ga_La,19130.0,0.951,0.3903,0.971341
9,GaSb_05kV_50pA,Sb,"[Ga, Sb]",Sb_La,986.0,0.049,0.6097,0.028659


In [56]:
# plotly plot

# fig = px.bar(df, x='Name', y='i/i_sum', color='Line', barmode='group', title='Relative intensities of lines with no correction')
# fig.update_xaxes(tickangle=90)
# fig.show()

In [57]:
# plotting stuff
x = df['x'].values
y = df['i/i_sum'].values
y2 = df['%at'].values
name = df['Name'].values
line = df['Line'].values
element = df['Element'].values

lines_list_plotting = ['Sb_La', 'As_Ka', 'Ga_Ka', 'Ga_La', 'As_La']
elements_list_plotting = ['As', 'Ga', 'Sb']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'] 

In [58]:
fig = go.Figure()

legends = []
for i in range(len(x)):
    col = colors[lines_list_plotting.index(line[i])]
    show_leg = False if line[i] in legends else True
    fig.add_scatter(x=[x[i]], y=[y[i]], mode='markers', name=f'{line[i]}', marker_color=col, legendgroup=line[i], showlegend=show_leg)
    fig.add_scatter(x=[x[i]], y=[y2[i]], mode='markers', name=f'AZ {line[i]}', marker_color=col, marker_symbol='x', legendgroup=line[i], showlegend=show_leg)
    legends.append(f'{line[i]}')

# x ticks are the filenames
fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), tickangle=90, range=[-0.5, len(df['x'].unique())-0.5])
fig.update_yaxes(range=[0.33, 0.67])
fig.update_layout(title='Relative intensities of lines with no correction', yaxis_title='Relative intensity')
fig.update_layout(height=600, width=1200)

'f'

'f'

In [59]:
## I / I_0 =  exp(-mu * (rrho*t)),
# where mu (or mu/rho) is the mass absorption coefficient (from HS)
# and rho is the density of the element (from HS, as the two densities added together and divided by 2)
# and t is the travel distance, estimated as the xray_range divided by 2 (also from HS, based on Anderson and Hasler (1966) 
# and I_0 is the generated intensity,
# and I is the measured intensity

# I_0 = I / exp(-mu * (rrho*t))

In [60]:
def get_mass_absorption_coeff_mixture(elements, line, weight_percent=[50,50]):
    """
    Returns the mass absorption coefficient for a mixture of elements.

    mass absorption coefficient(s) in cm^2/g

    Parameters
    ----------
    elements : list
        List of elements in the mixture.
    line : str
        The line to use for the calculation.
    weight_percent : list, optional
        The weight percent of each element in the mixture. The default is [50,50].

    Returns
    -------
    float
        The mass absorption coefficient for the mixture in cm^2/g.
    """
    return hs.material.mass_absorption_mixture(elements=elements, weight_percent=weight_percent, energies=line)


def get_density_mixture(elements, weight_percent=[50,50]):
    """
    Returns the density for a mixture of elements.

    Parameters
    ----------
    elements : list
        List of elements in the mixture.
    weight_percent : list, optional
        The weight percent of each element in the mixture. The default is [50,50].

    Returns
    -------
    float
        The density for the mixture.
    """
    d1 = hs.material.elements[elements[0]].Physical_properties.density_gcm3
    d2 = hs.material.elements[elements[1]].Physical_properties.density_gcm3
    return (d1*weight_percent[0] + d2*weight_percent[1])/100


# TODO: fix travel distance calculation to electron, not xrays


def get_path_length(elements:list, beam_energy: float, range_divider: float = 2, weight_percent: list = [50,50], angle: float = 35) -> float:
    """
    Crude assumnption that the path length (PL, t, Psi) is the electron range divided by 2.
    Kinda gets the average path length as the center of the interaction volume, but is not a good assumption.

    Parameters
    ----------
    elements : list
        List of elements in the mixture. Use the first element
    line : str
        The line to use for the calculation.
    beam_energy : float
        The beam energy in keV.
    weight_percent : list, optional
        The weight percent of each element in the mixture. The default is [50,50].
    angle : float, optional
        The take-off angle of the detector in degrees. The default is 35.

    Returns
    -------
    float
        The travel distance for the mixture, in cm.
    """
    um_to_cm = 1e-4
    rho = get_density_mixture(elements, weight_percent)
    electron_range = hs.eds.electron_range(elements[0], beam_energy, density=rho, tilt=0)
    # calculate PL = electron_range * csc(angle)
    # where csc(angle) = 1/sin(angle), and angle is in degrees
    path_length = electron_range / np.sin(np.deg2rad(angle))
    return path_length  * um_to_cm / range_divider



def calculate_i_gen(i, elements, line, beam_energy, weight_percent=[50,50], r_div=2):
    """
    Returns the generated intensity for a mixture of elements.

    i0 = i / exp(-mu * rho * t)

    Parameters
    ----------
    i : float
        The measured intensity.
    elements : list
        List of elements in the mixture.
    line : str
        The line to use for the calculation.
    beam_energy : float
        The beam energy in keV.
    weight_percent : list, optional
        The weight percent of each element in the mixture. The default is [50,50].

    Returns
    -------
    float
        The generated intensity for the mixture.
    """
    mu = get_mass_absorption_coeff_mixture(elements, line, weight_percent)
    rho = get_density_mixture(elements, weight_percent)
    t = get_path_length(elements, beam_energy, range_divider=r_div)
    return i / np.exp(-mu * rho * t)

In [61]:
# calculate mu, rho, t, i_0, and A (i=1), and add to df

mu_list = []
rho_list = []
t_list = []
A_list = []
i0_list = []
# i0_r3_list = []
# i0_r4_list = []
for j in range(len(df)):
    mu_list.append(round(get_mass_absorption_coeff_mixture(df['Elements'][j], df['Line'][j]), 1))
    rho_list.append(get_density_mixture(df['Elements'][j]))
    t_list.append(get_path_length(df['Elements'][j], df['kV'][j]))
    A_list.append(round(calculate_i_gen(i=1, elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j]), 2))
    i0_list.append(round(calculate_i_gen(i=df['i'][j], elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j]), 1))
    # i0_r3_list.append(round(calculate_i_gen(i=df['i'][j], elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j], r_div=3), 1))
    # i0_r4_list.append(round(calculate_i_gen(i=df['i'][j], elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j], r_div=4), 1))
df['mu'] = mu_list
# df['rho'] = rho_list
# df['t'] = t_list
df['A'] = A_list
df['i0'] = i0_list
# df['i0_r3'] = i0_r3_list
# df['i0_r4'] = i0_r4_list
df.head(3)

,File,Element,%at,Line,Name,i,i/i_sum,kV,x,at_wt,Elements,i: at%,mu,A,i0
0,GaAs_05kV_25pA,As,0.5342,As_La,GaAs 05kV 25pA,13128.0,0.445,5.0,0,74.9216,"[As, Ga]",0.427317,3744.1,1.56,20416.8
1,GaAs_05kV_25pA,Ga,0.4658,Ga_La,GaAs 05kV 25pA,16365.0,0.555,5.0,0,69.7230,"[As, Ga]",0.572683,1387.4,1.18,19274.6
2,GaAs_15kV_25pA,As,0.5062,As_La,GaAs 15kV 25pA,61699.0,0.374,15.0,1,74.9216,"[As, Ga]",0.357322,3744.1,15.89,980576.1


In [62]:
# calculate i0/i0_sum for each x, and add to df

def calculate_relative_intensity(column: str, df=df):
    """
    Returns the relative intensity for each x value in the dataframe.
    """
    relative_intensities = []
    for j in range(len(df)):
        i0 = df[column][j]
        i0_sum = df[df['x']==df['x'][j]][column].sum()
        # print(i0/i0_sum)
        relative_intensities.append(round(i0/i0_sum, 3))
    return relative_intensities

i0_sum_list = calculate_relative_intensity('i0')
# i0_r3_sum_list = calculate_relative_intensity('i0_r3')
# i0_r4_sum_list = calculate_relative_intensity('i0_r4')

df['i0/i0_sum'] = i0_sum_list
# df['i0_r3/i0_sum'] = i0_r3_sum_list
# df['i0_r4/i0_sum'] = i0_r4_sum_list
df.head(3)


,File,Element,%at,Line,Name,i,i/i_sum,kV,x,at_wt,Elements,i: at%,mu,A,i0,i0/i0_sum
0,GaAs_05kV_25pA,As,0.5342,As_La,GaAs 05kV 25pA,13128.0,0.445,5.0,0,74.9216,"[As, Ga]",0.427317,3744.1,1.56,20416.8,0.514
1,GaAs_05kV_25pA,Ga,0.4658,Ga_La,GaAs 05kV 25pA,16365.0,0.555,5.0,0,69.7230,"[As, Ga]",0.572683,1387.4,1.18,19274.6,0.486
2,GaAs_15kV_25pA,As,0.5062,As_La,GaAs 15kV 25pA,61699.0,0.374,15.0,1,74.9216,"[As, Ga]",0.357322,3744.1,15.89,980576.1,0.773


In [63]:
def calculate_atom_percent(column: str, df=df):
    """
    Returns the atom percent from the weight percent for each x value in the dataframe.
    """
    atom_percent_list = []
    for xi in df['x'].unique():
        wt_i0 = df[df['x']==xi][column].tolist()
        at_wt = df[df['x']==xi]['at_wt'].tolist()
        # at%_1 = (wt%_1 * at_wt_1) / (wt%_1 * at_wt_1 + wt%_2 * at_wt_2)
        atom_percent_list.append((wt_i0[0]/at_wt[0]) / (wt_i0[0]/at_wt[0] + wt_i0[1]/at_wt[1]))
        atom_percent_list.append((wt_i0[1]/at_wt[1]) / (wt_i0[0]/at_wt[0] + wt_i0[1]/at_wt[1]))
    return atom_percent_list

df['Elements'] = elements_list
df['i0: at%'] = calculate_atom_percent('i0/i0_sum')
df.head(3)

,File,Element,%at,Line,Name,i,i/i_sum,kV,x,at_wt,Elements,i: at%,mu,A,i0,i0/i0_sum,i0: at%
0,GaAs_05kV_25pA,As,0.5342,As_La,GaAs 05kV 25pA,13128.0,0.445,5.0,0,74.9216,"[As, Ga]",0.427317,3744.1,1.56,20416.8,0.514,0.496026
1,GaAs_05kV_25pA,Ga,0.4658,Ga_La,GaAs 05kV 25pA,16365.0,0.555,5.0,0,69.7230,"[As, Ga]",0.572683,1387.4,1.18,19274.6,0.486,0.503974
2,GaAs_15kV_25pA,As,0.5062,As_La,GaAs 15kV 25pA,61699.0,0.374,15.0,1,74.9216,"[As, Ga]",0.357322,3744.1,15.89,980576.1,0.773,0.760135


In [64]:
df[['File', 'Line', '%at', 'i/i_sum', 'i0/i0_sum','i0: at%', 'i: at%', 'at_wt']]

,File,Line,%at,i/i_sum,i0/i0_sum,i0: at%,i: at%,at_wt
0,GaAs_05kV_25pA,As_La,0.5342,0.445,0.514,0.496026,0.427317,74.9216
1,GaAs_05kV_25pA,Ga_La,0.4658,0.555,0.486,0.503974,0.572683,69.7230
2,GaAs_15kV_25pA,As_La,0.5062,0.374,0.773,0.760135,0.357322,74.9216
3,GaAs_15kV_25pA,Ga_La,0.4938,0.626,0.227,0.239865,0.642678,69.7230
4,GaAs_30kV_25pA,As_Ka,0.4841,0.373,0.418,0.400616,0.356341,74.9216
5,GaAs_30kV_25pA,Ga_Ka,0.5159,0.627,0.582,0.599384,0.643659,69.7230
6,GaAs_30kV_50pA,As_Ka,0.4831,0.372,0.417,0.399629,0.355361,74.9216
7,GaAs_30kV_50pA,Ga_Ka,0.5169,0.628,0.583,0.600371,0.644639,69.7230
8,GaSb_05kV_50pA,Ga_La,0.3903,0.951,0.967,0.980833,0.971341,69.7230
9,GaSb_05kV_50pA,Sb_La,0.6097,0.049,0.033,0.019167,0.028659,121.7600


In [65]:
fig = go.Figure()
legends = []
for i in range(len(x)):
    # col = colors[lines_list_plotting.index(line[i])]
    col = colors[elements_list_plotting.index(df['Element'][i])]
    showlegend = False if line[i] in legends else True
    marker_line_width = 1 if ("La" in line[i]) else 0
    fig.add_scatter(x=[x[i]], y=[df['i/i_sum'][i]], mode='markers', name=f'{line[i]}, i wt%', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, symbol='circle', line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[y2[i]], mode='markers', name=f'{line[i]}, AZ at%', marker_symbol='x', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[df['i0/i0_sum'][i]], mode='markers', name=f'{line[i]}, i0 wt%', marker_symbol='diamond', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    legends.append(f'{line[i]}')

# x ticks are the filenames
fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), tickangle=90, range=[-0.5, len(df['x'].unique())-0.5])
# fig.update_yaxes(range=[0.33, 0.67])
fig.update_layout(title='AZtec at% and wt% for raw intensity (i, circle) and A-corrected intensity (i0, diamonds)', yaxis_title='Relative intensity')
fig.update_layout(height=600, width=1200)
fig.update_layout(legend=dict(x=1.01, y=0.5, traceorder='normal', font=dict(family='sans-serif', size=12, color='black'))) 
fig.show()

In [66]:
fig = go.Figure()
legends = []
for i in range(len(x)):
    # col = colors[lines_list_plotting.index(line[i])]
    col = colors[elements_list_plotting.index(df['Element'][i])]
    showlegend = False if line[i] in legends else True
    marker_line_width = 1 if ("La" in line[i]) else 0
    fig.add_scatter(x=[x[i]], y=[df['i: at%'][i]], mode='markers', name=f'{line[i]}, i at%', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, symbol='circle', line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[y2[i]], mode='markers', name=f'{line[i]}, AZ at%', marker_symbol='x', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[df['i0: at%'][i]], mode='markers', name=f'{line[i]}, i0 at%', marker_symbol='diamond', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    legends.append(f'{line[i]}')

# x ticks are the filenames
fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), tickangle=90, range=[-0.5, len(df['x'].unique())-0.5])
# fig.update_yaxes(range=[0.33, 0.67])
fig.update_layout(title='AZtec at% and at% for i (circle) and i0 (diamonds)', yaxis_title='Relative intensity')
fig.update_layout(height=600, width=1200)
fig.update_layout(legend=dict(x=1.01, y=0.5, traceorder='normal', font=dict(family='sans-serif', size=12, color='black'))) 
fig.show()

In [67]:
# write a output file
df.to_csv('output.csv', index=False)

In [68]:
df

,File,Element,%at,Line,Name,i,i/i_sum,kV,x,at_wt,Elements,i: at%,mu,A,i0,i0/i0_sum,i0: at%
0,GaAs_05kV_25pA,As,0.5342,As_La,GaAs 05kV 25pA,13128.0,0.445,5.0,0,74.9216,"[As, Ga]",0.427317,3744.1,1.56,20416.8,0.514,0.496026
1,GaAs_05kV_25pA,Ga,0.4658,Ga_La,GaAs 05kV 25pA,16365.0,0.555,5.0,0,69.7230,"[As, Ga]",0.572683,1387.4,1.18,19274.6,0.486,0.503974
2,GaAs_15kV_25pA,As,0.5062,As_La,GaAs 15kV 25pA,61699.0,0.374,15.0,1,74.9216,"[As, Ga]",0.357322,3744.1,15.89,980576.1,0.773,0.760135
3,GaAs_15kV_25pA,Ga,0.4938,Ga_La,GaAs 15kV 25pA,103207.0,0.626,15.0,1,69.7230,"[As, Ga]",0.642678,1387.4,2.79,287622.7,0.227,0.239865
4,GaAs_30kV_25pA,As,0.4841,As_Ka,GaAs 30kV 25pA,83787.0,0.373,30.0,2,74.9216,"[As, Ga]",0.356341,123.4,1.34,111979.7,0.418,0.400616
5,GaAs_30kV_25pA,Ga,0.5159,Ga_Ka,GaAs 30kV 25pA,140854.0,0.627,30.0,2,69.7230,"[As, Ga]",0.643659,43.2,1.11,155907.6,0.582,0.599384
6,GaAs_30kV_50pA,As,0.4831,As_Ka,GaAs 30kV 50pA,179725.0,0.372,30.0,3,74.9216,"[As, Ga]",0.355361,123.4,1.34,240198.9,0.417,0.399629
7,GaAs_30kV_50pA,Ga,0.5169,Ga_Ka,GaAs 30kV 50pA,303060.0,0.628,30.0,3,69.7230,"[As, Ga]",0.644639,43.2,1.11,335449.2,0.583,0.600371
8,GaSb_05kV_50pA,Ga,0.3903,Ga_La,GaSb 05kV 50pA,19130.0,0.951,5.0,4,69.7230,"[Ga, Sb]",0.971341,3981.2,1.59,30364.1,0.967,0.980833
9,GaSb_05kV_50pA,Sb,0.6097,Sb_La,GaSb 05kV 50pA,986.0,0.049,5.0,4,121.7600,"[Ga, Sb]",0.028659,449.6,1.05,1038.8,0.033,0.019167


In [69]:
stop here to see the plot below!

SyntaxError: invalid syntax (1575662230.py, line 1)

In [ ]:
fig = go.Figure()

y1 = df['i/i_sum']
# y2 = df['i_gen/sum']
y2t2 = df['i_gen_t2/sum']
y2t3 = df['i_gen_t3/sum']
y2t4 = df['i_gen_t4/sum']

y3 = df['%at']

fig.add_trace(go.Scatter(x=x, y=y1, name='Measured Intensity', mode='markers'))
# fig.add_trace(go.Scatter(x=x, y=y2, name='A corrected Intensity', mode='markers'))
fig.add_trace(go.Scatter(x=x, y=y2t2, name='A corrected Intensity, t2', mode='markers'))
fig.add_trace(go.Scatter(x=x, y=y2t3, name='A corrected Intensity, t3', mode='markers'))
fig.add_trace(go.Scatter(x=x, y=y2t4, name='A corrected Intensity, t4', mode='markers'))

fig.add_trace(go.Scatter(x=x, y=y3, name='AZ comp', mode='markers'))

fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), tickangle=90, range=[-0.5, len(df['x'].unique())-0.5])


fig


fig.update_layout(title='Intensity with and without A correction', yaxis_title='Relative intensity')